In [1]:
from keras.models import load_model
from keras import backend as K
import pickle
import glob
import pandas as pd
import librosa
from keras.models import load_model
from keras import backend as K
import numpy as np
from python_speech_features import mfcc

C:\Users\D\anaconda3\envs\ml\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\D\anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\D\anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#設定test的wav路徑
wavs = glob.glob('D:\\Python\\ML\\HW3\\ML@NTUT-2021-Autumn-ASR\\test\\*.wav')
with open('dictionary.pkl', 'rb') as fr:
    [char2id, id2char, mfcc_mean, mfcc_std] = pickle.load(fr)

mfcc_dim = 13
###############################################
#loading model
model = load_model('asr4.h5',compile=False)
###############################################
def test_audio(wavs,index):
    '''index = np.random.randint(len(wavs))'''
    #print(wavs[index])
    path_name=wavs[index].split("\\")[-1]
    path_name=path_name.split(".")[0]
    audio, sr = librosa.load(wavs[index])
    energy = librosa.feature.rms(audio)
    frames = np.nonzero(energy >= np.max(energy) / 5)
    indices = librosa.core.frames_to_samples(frames)[1]
    audio = audio[indices[0]:indices[-1]] if indices.size else audio[0:0]
    X_data = mfcc(audio, sr, numcep=mfcc_dim, nfft=551)
    X_data = (X_data - mfcc_mean) / (mfcc_std + 1e-14)
    #print(X_data.shape)
    pred = model.predict(np.expand_dims(X_data, axis=0))
    pred_ids = K.eval(K.ctc_decode(pred, [X_data.shape[0]], greedy=False, beam_width=10, top_paths=1)[0][0])
    pred_ids = pred_ids.flatten().tolist()
    #print(''.join([id2char[i] for i in pred_ids]))
    
    # 使用CTC會使input與output長度一致
    # 導致output結尾會輸出許多-1
    # 須將其移除 才能正確進行decoder
    while(-1 in pred_ids): #將predict的-1移除
        pred_ids.remove(-1)
    return(path_name,''.join([id2char[i] for i in pred_ids])) #進行decoder


In [3]:
sol=[]
path_name=0
for i in range(len(wavs)):
    path_name , pre_data=test_audio(wavs,i)
    sol.append((path_name,pre_data))
df = pd.DataFrame(sol, columns = ['id' , 'text'])
# 重新進行id的排序
df.id = df.id.astype('int64')
df=df.sort_values(by='id')
# 儲存test的預測結果
df.to_csv("sub4.csv",index=False)
#觀察結果
print(df)

      id                                               text
0      1  tset ho tsan se kai bing pang kah tiat lin khu...
111    2                     ing am gua tsiah ka li tean lo
222    3                                     ah beh sian po
280    4                                  sC sit giam po so
291    5           a sui kam mo a pai to li the i tshing ka
..   ...                                                ...
270  342         i sing jin iau koh u tsin tng e lo ai kian
271  343                tsap ji pue a siak phoa kam iau kin
272  344  tit jit pun sa khu la khui e hit tang tsun khu...
273  345  bi kok kah Iran ma u kiong tong li ik pi lu Ir...
274  346  si lai tsi sin bunm bang tsa e phing lon kan t...

[346 rows x 2 columns]
